In [18]:
from books.models import Publisher
from django_prepared_query import BindParam

prepared_qs = Publisher.prepared_objects.filter(id=BindParam('id')).prepare()
qs = Publisher.objects.all()
%timeit prepared_qs.execute(id=160)
%timeit list(qs.filter(id=160))

524 µs ± 68.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
661 µs ± 78.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [20]:
from books.models import Book

prepared_qs = Book.prepared_objects.filter(authors__age=BindParam('author_age'), pages__lte=BindParam('max_pages'), 
                                           pages__gte=BindParam('min_pages')).prepare()
qs = Book.objects.all()
%timeit prepared_qs.execute(author_age=35, min_pages=100, max_pages=1000)
%timeit list(qs.filter(authors__age=35, pages__gte=100, pages__lte=1000))

940 µs ± 137 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.12 ms ± 246 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
prepared_qs = Book.prepared_objects.select_related('publisher').filter(id=BindParam('id')).prepare()
qs = Book.objects.select_related('publisher')
%timeit prepared_qs.execute(id=160)
%timeit list(qs.filter(id=160))

961 µs ± 92.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.24 ms ± 153 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [18]:
from django.db.models import Count
prepared_qs = Book.prepared_objects.annotate(num_of_authors=Count('authors')).filter(id=160).prepare()
qs = Book.objects.annotate(num_of_authors=Count('authors')).filter(id=160)
%timeit prepared_qs.execute()
%timeit list(qs.all())

690 µs ± 50.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.02 ms ± 97.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
prepared_qs = Book.prepared_objects.prefetch_related('authors').filter(id__in=[160, 161]).prepare()
qs = Book.objects.prefetch_related('authors').filter(id__in=[160, 161])
%timeit prepared_qs.execute()
%timeit list(qs.all())

3.18 ms ± 562 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.78 ms ± 526 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
prepared_qs = Book.prepared_objects.all()[0:5].prepare()
qs = Book.objects.all()[0:5]
%timeit prepared_qs.execute()
%timeit list(qs.all())

711 µs ± 57 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
731 µs ± 44.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [25]:
from django.db.models import Count
prepared_qs = Book.prepared_objects.annotate(num_of_authors=Count('authors')). \
    filter(num_of_authors__gte=BindParam('num_of_authors')).prepare()
qs = Book.objects.annotate(num_of_authors=Count('authors'))
%timeit prepared_qs.execute(num_of_authors=6)
%timeit list(qs.filter(num_of_authors__gte=6))

2.17 ms ± 248 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.5 ms ± 282 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
